Libraries that I need to install

In [1]:
!pip install bert-for-tf2
!pip install sentencepiece
!pip install pdpipe 
!pip install symspellpy
!pip install pycontractions

     |████████████████████████████████| 40 kB 1.8 MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.13.5-py3-none-any.whl size=29946 sha256=854ccbf000fc6a917c7298b565bc2ef82c0871ed9835daf49a5be28d504d30b9
  Stored in directory: /root/.cache/pip/wheels/90/00/4a/e3ae98003e155850dee523617ced7794e6e3da07d27e3802a0
  Created wheel for py-params: filename=py_params-0.8.3-py3-none-any.whl size=4690 sha256=a7a01a7ebe863c7b4c9e16f51cde1db7e105613e601c9e1b3b72788b85799953
  Stored in directory: /root/.cache/pip/wheels/61/14/29/7adda9e71ff1d1aff49809918f7d37d8325683ced69860c527
  Created wheel for params-flow: filename=params_flow-0.7.4-py3-none-any.whl size=16194 sha256=8acb8b351542cd08689f254b4273bac356116e126ec2dc4e5d3f2fec0d0027b3
  Stored in directory: /root/.cache/pip/wheels/f4/90/ca/a5d500c088762773489e532acd0268226cf2597f211928f38a
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 47 kB 1.0 MB/s 
  Created wheel for decore: filenam

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam



########################################################
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub



try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert


#data pipeline 
import pdpipe as pdp

#Other Preprocessing
import re
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
import string
# !pip install symspellpy
import pkg_resources
from symspellpy.symspellpy import SymSpell
from symspellpy import SymSpell, Verbosity
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import csv
import pandas as pd

import nltk
nltk.download('punkt')

#Contraction Import
from pycontractions import Contractions

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


Using TensorFlow backend.


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Reading the text and storing it... 

In [3]:
train= pd.read_csv('../input/nlp-getting-started/train.csv')
test=pd.read_csv('../input/nlp-getting-started/test.csv')
train.head(3)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1


## Web Scrapping done for Abbreviations and then preprocessed
Version Edit: Anit G.
* Removed Abusive words
* Removed empty slangs
* Removed slangs such as "?- it means some body is asking question"
* Removed slangs with length more than 7

In [4]:
from bs4 import BeautifulSoup
import requests, json, csv
import re

def remove_wrong_abb(key,value):
    if(re.search('it means',value)):
        return False;
    elif(re.search('\*',value)):
        return False
    elif(value==""):
        return False
    elif(len(key)>7):
        return False
    else :
        return True
    
resp = requests.get("http://www.netlingo.com/acronyms.php")
soup = BeautifulSoup(resp.text, "html.parser")
slangdict= {}
key=""
value=""
for div in soup.findAll('div', attrs={'class':'list_box3'}):
    for li in div.findAll('li'):
        for a in li.findAll('a'):
            key =a.text
            value = li.text.split(key)[1]
            if(remove_wrong_abb(key,value)):
                if(re.search('\ -or- ',value)):
                    pos=re.search('\ -or- ',value).start()
                    slangdict[key]=value[:pos]
                else:
                    slangdict[key]=value

                    
# with open('myslang.json', 'w') as f:
#     json.dump(slangdict, f, indent=2)

w = csv.writer(open("myslang.csv", "w"))
for key, val in slangdict.items():
    w.writerow([key.lower(), val.lower()])
    
reader = csv.reader(open('myslang.csv', 'r'))
abbreviations = {}
for row in reader:
    k, v = row
    abbreviations[k] = v


## All Preprocessing Configuration and Methods 

## SymSpell - Unbeatable Spell Checker

**Configuration Setup**

In [5]:
#Version Edit: Sonam D.
sym_spell_4space = SymSpell(max_dictionary_edit_distance=0, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell_4space.load_dictionary(dictionary_path, term_index=0, count_index=1)


sym_spell_misspelled = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
bigram_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_bigramdictionary_en_243_342.txt")
sym_spell_misspelled.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell_misspelled.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

True

## Google Trained for Contraction handling - pycontractions

**Configuration Setup**
[](http://)

In [6]:
cont = Contractions(api_key="glove-twitter-100")
cont.load_models()


[=====---------------------------------------------] 11.8% 45.8/387.1MB downloaded

## Pre processing  Methods

In [7]:

#Version Edit: Sonam D.
def to_lower(text):
    text = text.lower()
    return text

#Version Edit: Anit G.
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r' ',text)

#Version Edit: Anit G.
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r' ',text)

#Version Edit: Anit G.
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', text)

#Version Edit: Sonam D.
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

#Version Edit: Sonam D.
def remove_numbers(text):
    text = re.sub(r'\d+', ' ', text)
    return text

#Version Edit: Sonam D.
def remove_mentions(text):
    text = re.sub(r'@\w*', ' ', text)
    return text

#Version Edit: Sonam D.
def remove_punctuations(text):
    text = re.sub(r'([^A-Za-z \t])|(\w+:\/\/\S+)', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_square_bracket(text):
    text = re.sub('\[.*?\]', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_angular_bracket(text):
    text = re.sub('<.*?>+', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_newline(text):
    text = re.sub('\n', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_words_with_numbers(text):
    text = re.sub('\w*\d\w*', ' ', text)
    return text
    
#Version Edit: Sonam D.
def hashtag_to_words(text):
    text = re.sub(r'##', '#', text)
    hash_pattern = re.compile(r"#\w*")
    hashtag_list = re.findall(hash_pattern,text)
    for hashtag in hashtag_list:
        hashtag = re.sub(r'#', '', hashtag)
        text = re.sub(hashtag, sym_spell_4space.word_segmentation(hashtag).corrected_string, text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'# ', '', text)
    return text

#Version Edit: Susanth D.
def remove_stopwords(text):
    text_tokens=word_tokenize(text)
    textop = ''
    for token in text_tokens:
        if token not in stopwords.words('english'):
            textop = textop + token + ' '
    return textop

#Version Edit: Sonam D.
def correct_misspelled_with_context(text):
    suggestions = sym_spell_misspelled.lookup_compound(text, max_edit_distance=2)
    text = str(suggestions[0])
    text = re.sub(r', \d', ' ', text)
#     text = remove_numbers(text)
    return text

#Version Edit: Sonam D.
def stemming_text(text):
    stemmer= PorterStemmer()
    text_tokens=word_tokenize(text)
    textop = ''
    for token in text_tokens:
        textop = textop + stemmer.stem(token) + ' '
    return textop

#Version Edit: Sonam D.
def lemmatization(text):
    lemmatizer=WordNetLemmatizer()
    text_tokens=word_tokenize(text)
    textop = ''
    for token in text_tokens:
        textop = textop + lemmatizer.lemmatize(token) + ' '
    return textop

#Version Edit: Anit G.
def convert_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

#Version Edit: Anit G.
def convert_abbrev_in_text(text):
    tokens = word_tokenize(text)
    tokens = [convert_abbrev(word) for word in tokens]
    text = ' '.join(tokens)
    return text

#Version Edit: Sonam D.
# def handle_contractions(text):
#     text = re.sub(r"’", "'", text)
#     for word in text.split():
#         if word.lower() in contractions:
#             text = text.replace(word, contractions[word.lower()])
#     return(text)

#Version Edit: Saurabh M.
def removeRepeated(tweet):
    prev = ''
    tweet_new = ''
    for c in tweet:
        caps = False
        if c.isdigit():
            tweet_new += c
            continue
        if c.isalpha() == True:
            if ord(c) >= 65 and ord(c)<=90:
                caps = True
            c = c.lower()
            if c == prev:
                count += 1
            else:
                count = 1
                prev = c
            if count >= 3:
                continue
            if caps == True:
                tweet_new += c.upper()
            else:
                tweet_new += c
        else:
            tweet_new += c
    return tweet_new


def Expand_Contractions(text):
    return list(cont.expand_texts([text]))[0]

## Pre-processing Data Pipeline 


In [8]:
# pipeline=pdp.ApplyByCols('text',handle_contractions,'text')
# pipeline+=pdp.ApplyByCols('text',remove_URL,'text')
# pipeline+=pdp.ApplyByCols('text',remove_html,'text')
# pipeline+=pdp.ApplyByCols('text',remove_mentions,'text')
# pipeline+=pdp.ApplyByCols('text',remove_newline,'text')
# pipeline+=pdp.ApplyByCols('text',remove_emoji,'text')
# #pipeline+=pdp.ApplyByCols('text',hashtag_to_words,'text')
# pipeline+=pdp.ApplyByCols('text',remove_punctuations,'text')
# pipeline+=pdp.ApplyByCols('text',removeRepeated,'text')
# pipeline+=pdp.ApplyByCols('text',convert_abbrev_in_text,'text')
# pipeline+=pdp.ApplyByCols('text',lemmatization,'text')
# pipeline+=pdp.ApplyByCols('text',remove_stopwords,'text')


pipeline=pdp.ApplyByCols('text',to_lower,'text')
pipeline+=pdp.ApplyByCols('text',remove_newline,'text')
pipeline+=pdp.ApplyByCols('text',remove_URL,'text')
pipeline+=pdp.ApplyByCols('text',remove_html,'text')
pipeline+=pdp.ApplyByCols('text',remove_emoji,'text')
#pipeline+=pdp.ApplyByCols('text',hashtag_to_words,'text')
pipeline+=pdp.ApplyByCols('text',remove_words_with_numbers,'text')
pipeline+=pdp.ApplyByCols('text',remove_numbers,'text')
pipeline+=pdp.ApplyByCols('text',remove_mentions,'text')
pipeline+=pdp.ApplyByCols('text',remove_square_bracket,'text')
pipeline+=pdp.ApplyByCols('text',remove_angular_bracket,'text')
pipeline+=pdp.ApplyByCols('text',Expand_Contractions,'text')
pipeline+=pdp.ApplyByCols('text',remove_punctuations,'text')
pipeline+=pdp.ApplyByCols('text',removeRepeated,'text')
pipeline+=pdp.ApplyByCols('text',convert_abbrev_in_text,'text')
pipeline+=pdp.ApplyByCols('text',correct_misspelled_with_context,'text')
pipeline+=pdp.ApplyByCols('text',remove_numbers,'text')
pipeline+=pdp.ApplyByCols('text',remove_stopwords,'text')
#pipeline+=pdp.ApplyByCols('text',stemming_text,'text')
pipeline+=pdp.ApplyByCols('text',lemmatization,'text')




# df=pd.concat([train,test],sort=False)
train=pipeline(train)
test=pipeline(test)
test.head()

,id,keyword,location,text
0,0,NaN,NaN,happened terrible car crash
1,2,NaN,NaN,heard earthquake different city stay safe ever...
2,3,NaN,NaN,forest financial independence retire early spo...
3,9,NaN,NaN,apocalypse lighting spokane wildfire
4,11,NaN,NaN,typhoon decor kill china taiwan


## Bert Model [Implementation 1]-  Methods

In [9]:
"""BERT Methods Predefined"""
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

def build_model(bert_layer,learning_rate=2e-5,Dropout_num=0, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    
    if Dropout_num == 0:
        # Without Dropout
        out = Dense(1, activation='sigmoid')(clf_output)
    else:
        # With Dropout(Dropout_num), Dropout_num > 0
        x = Dropout(Dropout_num)(clf_output)
        out = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## Usuall Preprocessing Methods Not Compatible with pipelines.


In [10]:
# df['text']=df['text'].apply(lambda x : remove_URL(x))
# df['text']=df['text'].apply(lambda x : remove_html(x))
# df['text']=df['text'].apply(lambda x: remove_emoji(x))
# df['text']=df['text'].apply(lambda x : remove_punct(x))
# df["text"] = df["text"].apply(lambda x: convert_abbrev_in_text(x))
# #Case Correction
# #Spell Checker
# #Stop Words Remover
# #Abbreviation remover
# #Missing Spaces
# #Case Correction (Althoug we cant just do this. This will change the context)
# df.head(5)

In [11]:
print('There are {} rows and {} columns in train'.format(train.shape[0],train.shape[1]))
print('There are {} rows and {} columns in train'.format(test.shape[0],test.shape[1]))

There are 7613 rows and 5 columns in train
There are 3263 rows and 4 columns in train


# Bert Implementation

### Downloading a pretrained model

In [12]:
# Load BERT from the Tensorflow Hub

module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
# module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
#https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1
#https://tfhub.dev/tensorflow/bert_en_wwm_uncased_L-24_H-1024_A-16/1
bert_layer = hub.KerasLayer(module_url, trainable=True)




### **Tokenising Tweets**

In [13]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocab_file, do_lower_case)

In [14]:
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)
train_labels = train.target.values

## Model Implementation 1

**Version Edit : Anit G.**

In [15]:
model_BERT = build_model(bert_layer,learning_rate=9e-6,Dropout_num=0, max_len=160)
model_BERT.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [16]:
# Anit's model

checkpoint = ModelCheckpoint('model_BERT.h5', monitor='val_loss', save_best_only=True)


train_history = model_BERT.fit(
    train_input, train_labels,
    validation_split = 0.1,
    epochs = 3, # recomended 3-5 epochs
    callbacks=[checkpoint],
    batch_size = 16 # recomended 8-16
)

Train on 6851 samples, validate on 762 samples
Epoch 1/3
6851/6851 [==============================] - 476s 69ms/sample - loss: 0.4717 - accuracy: 0.7831 - val_loss: 0.4022 - val_accuracy: 0.8281
Epoch 2/3
6851/6851 [==============================] - 403s 59ms/sample - loss: 0.3589 - accuracy: 0.8527 - val_loss: 0.4228 - val_accuracy: 0.8189
Epoch 3/3
6851/6851 [==============================] - 403s 59ms/sample - loss: 0.2829 - accuracy: 0.8883 - val_loss: 0.5398 - val_accuracy: 0.7979


## Model Implementation 2
**Version Edit : Sushant D. **

In [17]:
# from tensorflow.keras.optimizers import SGD
# from keras.callbacks import EarlyStopping
# from keras import regularizers

In [18]:
# learning_rate=1e-7
# Dropout_num=0
# max_len=160

In [19]:
# input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
# input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
# segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
# _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
# clf_output = sequence_output[:, 0, :]

# ## Type1
# # out1 = Dense(784, activation='relu', kernel_regularizer=regularizers.l2(9e-5))(clf_output)
# # out2 = Dense(100, activation='relu', kernel_regularizer=regularizers.l2(9e-5))(out1)

# ## Type2
# out2 = Dense(768, activation='relu', activity_regularizer=regularizers.l2(9e-5))(clf_output)
# out = Dense(1, activation='sigmoid')(out2)

In [20]:
# sBERT = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
# # sBERT.compile(SGD(lr=learning_rate, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])
# sBERT.compile(Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
# sBERT.summary()

In [21]:
# early_stopping = EarlyStopping(monitor='val_accuracy',
#                                min_delta=0,
#                                patience=3,
#                                verbose=1, 
#                                mode='auto')

# checkpoint = ModelCheckpoint('sBERT.h5',
#                              monitor='val_loss',
#                              save_best_only=True)


# train_history = sBERT.fit(
#     train_input, train_labels,
#     validation_split = 0.3,
#     epochs = 10, # recomended 3-5 epochs
#     callbacks=[checkpoint,early_stopping],
#     batch_size = 32 # recomended 8-16
# )

In [22]:
sub = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [23]:
# # Susanth's Full model
# test_pred = sBERT.predict(test_input)
# sub['target'] = test_pred.round().astype(int)
# sub.to_csv('submission full.csv', index=False)
# sub.head(3)

In [24]:
# ## Susanth's checkpoint model
# sBERT.load_weights('sBERT.h5')
# test_pred = sBERT.predict(test_input)
# sub['target'] = test_pred.round().astype(int)
# sub.to_csv('submission checkpoint.csv', index=False)
# sub.head(3)


## Tuning Parameters Tried and Respective Accuracies [MI-2]

In [25]:
################## Tune Parameter 1 ##################
# Dropout_num = 0
# learning_rate = 1e-7
# valid = 0.3
# epochs_num = 10
# batch_size_num = 32
# Dense Layer = 1(Final)
# Val_accuracy : 
# Test_accuracy : 
######################################################
################## Tune Parameter 2 ##################
#Dropout_num = 0
#learning_rate = 9e-6
#valid = 0.2
#epochs_num = 5
#batch_size_num = 32
#Val_accuracy : 83.45
######################################################
################## Tune Parameter 3 ##################
#Dropout_num = 0
#learning_rate = 9e-6
#valid = 0.1
#epochs_num = 3
#batch_size_num = 16
#Val_accuracy : 
######################################################
################## Tune Parameter 4 ##################
#Dropout_num = 0
#learning_rate = 5e-6
#valid = 0.1
#epochs_num = 3
#batch_size_num = 16
#Val_accuracy : 81.63
######################################################
################## Tune Parameter 5 ##################
#Dropout_num = 0
#learning_rate = 5e-6
#valid = 0.1
#epochs_num = 5
#batch_size_num = 16
#Val_accuracy : 
######################################################
################## Tune Parameter 6 ##################
#Dropout_num = 0
#learning_rate = 9e-6
#valid = 0.1
#epochs_num = 5
#batch_size_num = 16
#Val_accuracy : 
######################################################

In [26]:
# # Prediction by BERT model with my tuning
# model_BERT.load_weights('model_BERT.h5')
# test_pred_BERT = model_BERT.predict(test_input)
# test_pred_BERT_int = test_pred_BERT.round().astype('int')
# # Prediction by BERT model with my tuning for the training data - for the Confusion Matrix
# train_pred_BERT = model_BERT.predict(train_input)
# train_pred_BERT_int = train_pred_BERT.round().astype('int')

## Generating Submission File with MI-1
**Version Edit: Anit G.**

In [27]:

model_BERT.load_weights('model_BERT.h5')
test_pred = model_BERT.predict(test_input)
sub['target'] = test_pred.round().astype(int)
sub.to_csv('submission.csv', index=False)
sub.head(3)

,id,target
0,0,1
1,2,1
2,3,1


## Generating Submission File with MI-2
**Version Edit: Susanth D.**